## 准备数据

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [4]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [5]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([784, 128]), dtype=tf.float32)
        self.b1 = tf.Variable(tf.zeros([128]), dtype=tf.float32)
        self.W2 = tf.Variable(tf.random.normal([128, 10]), dtype=tf.float32)
        self.b2 = tf.Variable(tf.zeros([10]), dtype=tf.float32)
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])  # 展平输入图像
        hidden = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(hidden, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [6]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [7]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 111.4768 ; accuracy 0.08476666
epoch 1 : loss 92.83283 ; accuracy 0.093666665
epoch 2 : loss 82.77393 ; accuracy 0.1022
epoch 3 : loss 76.35255 ; accuracy 0.11011667
epoch 4 : loss 71.7122 ; accuracy 0.11761667
epoch 5 : loss 68.07933 ; accuracy 0.1245
epoch 6 : loss 65.08528 ; accuracy 0.13123333
epoch 7 : loss 62.560284 ; accuracy 0.13768333
epoch 8 : loss 60.40606 ; accuracy 0.14403333
epoch 9 : loss 58.540527 ; accuracy 0.15073334
epoch 10 : loss 56.89892 ; accuracy 0.15648334
epoch 11 : loss 55.43557 ; accuracy 0.1626
epoch 12 : loss 54.115456 ; accuracy 0.16771667
epoch 13 : loss 52.90796 ; accuracy 0.17261666
epoch 14 : loss 51.78844 ; accuracy 0.17815
epoch 15 : loss 50.737392 ; accuracy 0.18238333
epoch 16 : loss 49.738667 ; accuracy 0.18735
epoch 17 : loss 48.78341 ; accuracy 0.19131666
epoch 18 : loss 47.865356 ; accuracy 0.19611667
epoch 19 : loss 46.979794 ; accuracy 0.20035
epoch 20 : loss 46.12251 ; accuracy 0.20451666
epoch 21 : loss 45.290333 ; accuracy 